### What was the biggest pitfall that eveyone did?

It was data leakage. When I examined the public notebooks, I saw a lot of people did fall into this trap. Me? Don't ask me. When compare the algorithms, I get good results. However, i forgot some algorithms' random state 😅.

Last but least, i think tuning were not effective because of imbalanced dataset. How do I know? I studied an extremely imbalaned dataset. But I can't say I'm so sure. Can't see the winners notebooks. You can reach out the notebook i mentioned here: https://www.kaggle.com/code/abdullahkavakli/credit-card-fraud-detection

In [ ]:

from itertools import combinations
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import xgboost as xgb
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from mlxtend.classifier import StackingCVClassifier
from scipy.stats import uniform, randint
from sklearn.impute import KNNImputer
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, accuracy_score, average_precision_score, make_scorer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, KFold, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import RobustScaler, power_transform, OneHotEncoder, PowerTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from imblearn.ensemble import BalancedBaggingClassifier, EasyEnsembleClassifier, RUSBoostClassifier, BalancedRandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.svm import SVC
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import Perceptron
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.svm import NuSVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from imblearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from imblearn.combine import *
from imblearn.over_sampling import *
from imblearn.under_sampling import *
import hyperopt 
import sys
import copy
from hyperopt.pyll import scope
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from warnings import filterwarnings

filterwarnings("ignore")
np.random.seed(42)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#submission = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv")


In [ ]:
df_train = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
df_test = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
df_greeks = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/greeks.csv")

print('No. of records for train : {}'.format(df_train.shape))
print('No. of records for test : {}'.format(df_test.shape))
print('No. of records for greeks : {}'.format(df_greeks.shape))

In [ ]:
rand_state=42

In [ ]:
def balanced_logloss(y_true, y_pred):
    if isinstance(y_true, pd.DataFrame):
        y_true = y_true.to_numpy().ravel()
    if len(y_true.shape) == 2:
        y_true = y_true.ravel()
    nc = np.bincount(y_true)
    return log_loss(y_true, y_pred, sample_weight = 1/nc[y_true], eps=1e-15)

In [ ]:
def contains_none(data_x):
    if isinstance(data_x, pd.DataFrame):
        return data_x.isna().any().any()
    if isinstance(data_x, np.ndarray):
        return np.isnan(data_x).any()
    raise Exception("Unknown data type")

In [ ]:
def impute_train_test(train, test, impute_method="knn"):
    if impute_method == "knn":
        imputer = KNNImputer().set_output(transform="pandas")
        train = imputer.fit_transform(train)
        test = imputer.transform(test)
    if impute_method == "lgbm":
        imputer = LGBMImputer()
        train = imputer.fit_transform(train)
        test = imputer.transform(test)
    if contains_none(train) or contains_none(test): #if lgbm didn't impute, failed
        train, test = impute_train_test(train, test, impute_method="knn")
    return train,test    

In [ ]:
def transform_and_scale(train, test, power_transform=False):
    if power_transform:
        pt = PowerTransformer().set_output(transform="pandas")
        train = pt.fit_transform(train)
        test = pt.transform(test)
    scaler = RobustScaler().set_output(transform="pandas")
    train_scaled = scaler.fit_transform(train)
    test_scaled = scaler.transform(test)
    return train_scaled, test_scaled

In [ ]:
def process_train_test(X_train, X_test, impute_method="knn", power_transform=False, one_hot_encode=True):
        numeric_columns = X_train.select_dtypes([float,int]).columns
        categorical_columns = X_train.select_dtypes(object).columns
        
        X_train.loc[:,numeric_columns], X_test.loc[:,numeric_columns] =\
        transform_and_scale(X_train.loc[:,numeric_columns],
                            X_test.loc[:,numeric_columns], 
                            power_transform=power_transform)
        if not categorical_columns.empty:
            le = LabelEncoder()
            le.fit(X_train.loc[:,categorical_columns])#changed in 5.0.0dev2
            X_train.loc[:,categorical_columns] = le.transform(X_train.loc[:,categorical_columns])
            X_test.loc[:,categorical_columns] = le.transform(X_test.loc[:,categorical_columns])
        if contains_none(X_train) or contains_none(X_test): #changed in 5.0.0dev2
            X_train, X_test = impute_train_test(X_train, X_test, impute_method=impute_method)      
        if one_hot_encode and (not categorical_columns.empty):#çıkacak mı sorun bi bak
            enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform="pandas")
            X_train = pd.concat([X_train, enc.fit_transform(X_train.loc[:,categorical_columns])] ,axis=1)
            X_test = pd.concat([X_test, enc.transform(X_test.loc[:,categorical_columns])] ,axis=1)
            
        #X_train = pd.get_dummies(X_train, columns=categorical_columns, dtype=int)
        #X_test = pd.get_dummies(X_test, columns=categorical_columns, dtype=int) 
        return X_train, X_test

In [ ]:
b_log_loss_scorer = make_scorer(balanced_logloss, greater_is_better=False)

In [ ]:
X_raw = df_train.drop(columns=["Id", "Class"]).copy(deep=True)
y = df_train.loc[:,["Class"]].copy(deep=True)
X_test_raw = df_test.drop(columns=["Id"])
X_train, X_test = process_train_test(X_raw, X_test_raw)

X_us, y_us = CondensedNearestNeighbour(random_state=rand_state,n_jobs=-1).fit_resample(X_train, y) 

clf_meta = LGBMClassifier(boosting_type='goss', random_state=42, verbose=-1,
                                           is_unbalance=True, n_jobs=-1)
classifiers_stack=[("XGB Random Forest Classifier", xgb.XGBRFClassifier(objective="binary:logistic", eval_metric=b_log_loss_scorer, 
                                                                       tree_method="hist", random_state=rand_state,scale_pos_weight=4.71,
                                                                       enable_categorical=True)),
                   ("Cat Boost", CatBoostClassifier(verbose=0, objective="Logloss", auto_class_weights='Balanced', thread_count=-1)),
                   ("LGBM", LGBMClassifier(boosting_type='goss', random_state=42, verbose=-1,
                                           is_unbalance=True, n_jobs=-1)),
                   ("QDA", QuadraticDiscriminantAnalysis(priors=[0.5, 0.5])),
                   ("LDA", LinearDiscriminantAnalysis(priors=[0.5, 0.5])),
                  ]
clf_stack = StackingClassifier(estimators=classifiers_stack, 
                               stack_method="predict_proba",
                               passthrough=False,
                               final_estimator=clf_meta)
model =clf_stack 
model.fit(X_us, y_us.to_numpy())

prob_all = model.predict_proba(X_test)
cls_0 = prob_all[:,0]
cls_1 = prob_all[:,1]
submission = df_test.loc[:,["Id"]].copy(deep=True)
submission["class_0"] = cls_0
submission["class_1"] = cls_1
print(prob_all)

In [ ]:
submission

In [ ]:
submission.to_csv("/kaggle/working/submission.csv", index=False)